# Fig2

## Fig2.A

### Integrated UMAP
cell_info.ci: Generated by `preprocesses/5.annotation/cell_annotation_process.ipynb`

In [1]:
pacman::p_load(Seurat, SeuratDisk, tidyverse, plyr)
seurat <-
    LoadH5Seurat(
        "../resources/__integrated.h5seurat"
    )
seurat@meta.data <-
    seurat@meta.data %>%
    tibble::rownames_to_column("cell") %>%
    select(-sample) %>%
    left_join(
        read_tsv(
            "../resources/cellinfo.ci",
            show_col_types = FALSE
        ),
        by = "cell"
    ) %>%
    mutate(
        sample = factor(
            sample,
            levels = read_tsv(
                "../meta/sample_metadata.tsv",
                show_col_types = FALSE
            ) %>%
                distinct(sample) %>%
                pull()
        ),
        celltype = factor(
            celltype,
            levels = read_tsv(
                "../meta/cell_metadata.tsv",
                show_col_types = FALSE
            ) %>%
                distinct(celltype) %>%
                pull()
        )
    ) %>%
    tibble::column_to_rownames("cell")
seurat %>%
    SingleCellPipeline::plot_umap(
        raster = FALSE
    ) %>%
    SingleCellPipeline::ggsave_stream(
        "../figs/fig1A1.png",
        width = 22,
        height = 20,
        units = "cm"
    )

### Each sample UMAP
cell_info.ci: Generated by `preprocesses/5.annotation/cell_annotation_process.ipynb`

In [ ]:
read_tsv(
    "../meta/sample_metadata.tsv",
    show_col_types = FALSE
) %>%
    distinct(sample) %>%
    pull() %>%
    as.list() %>%
    lapply(
        function(target_sample) {
            plt <-
                LoadH5Seurat(
                    "../resources/",
                    target_sample,
                    ".h5seurat"
                ) %>%
                SingleCellPipeline::assign_celltype(
                    annotation = read_tsv(
                        "../meta/annotations/",
                        target_sample,
                        ".tsv"
                    ),
                    cell.meta = read_tsv(
                        "../meta/cell_metadata.tsv",
                        show_col_types = FALSE
                    ),
                    sample.meta = read_tsv(
                        "../meta/sample_metadata.tsv",
                        show_col_types = FALSE
                    )
                ) %>%
                SingleCellPipeline::plot_umap(
                    raster = FALSE
                )
            plt <-
                plt + ggtitle(target_sample)
            return(plt)
        }
    ) %>%
    patchwork::wrap_plots() %>%
    SingleCellPipeline::ggsave_stream(
        "../figs/fig1A2.png",
        width = 50,
        height = 30,
        units = "cm"
    )

## Fig2.B

In [ ]:
read_tsv(
    "../meta/sample_metadata.tsv",
    show_col_types = FALSE
) %>%
    distinct(sample) %>%
        pull() %>%
        as.list() %>%
        lapply(
            function(target_sample) {
                seurat <-
                    LoadH5Seurat(
                        paste0(
                            "../resources/",
                            target_sample,
                            ".h5seurat"
                        )
                    ) %>%
                    SingleCellPipeline::assign_celltype(
                        annotation = read_tsv(
                            "../meta/annotations/",
                            target_sample,
                            ".tsv"
                        ),
                        cell.meta = read_tsv(
                            "../meta/cell_metadata.tsv",
                            show_col_types = FALSE
                        ),
                        sample.meta = read_tsv(
                            "../meta/sample_metadata.tsv",
                            show_col_types = FALSE
                        )
                    )
                markers <-
                    seurat@meta.data %>%
                    distinct(celltype, marker, color.celltype) %>%
                    arrange(celltype) %>%
                    mutate(
                        marker = str_split(marker, ",")
                    ) %>%
                    unnest(marker)
                expression <-
                    seurat %>%
                    SeuratObject::FetchData(
                        vars = markers %>%
                            distinct(marker) %>%
                            filter(
                                !is.na(marker)
                            ) %>%
                            pull()
                    ) %>%
                    tibble::rownames_to_column("cell") %>%
                    pivot_longer(
                        cols = -cell,
                        names_to = "gene",
                        values_to = "expression"
                    )
                plt <-
                    seurat %>%
                    DotPlot(
                        assay = "SCT",
                        features = markers %>%
                            filter(
                                !is.na(marker)
                            ) %>%
                            distinct(marker) %>%
                            pull(),
                        col.min = 0
                    ) +
                    coord_flip() +
                    theme(
                        axis.text.x.bottom = element_text(
                            angle = 45,
                            hjust = 1,
                            color = markers %>%
                                arrange(celltype) %>%
                                distinct(color.celltype) %>%
                                pull()
                        ),
                        axis.text.y = element_text(
                            colour = markers %>%
                                filter(
                                    marker %in% (
                                        expression %>%
                                            distinct(gene) %>%
                                            pull()
                                    )
                                ) %>%
                                select(color.celltype) %>%
                                pull()
                        ),
                        legend.justification = "bottom",
                        plot.background = element_rect(
                            fill = "white",
                            color = "white"
                        )
                    ) +
                    scale_color_gradient2(
                        low = "gray",
                        high = assigned@meta.data %>%
                            distinct(color.sample) %>%
                            pull()
                    ) +
                    labs(
                        x = "Marker",
                        y = "Cell type"
                    )
                return(plt)
            }
        ) %>%
        patchwork::wrap_plots() %>%
        SingleCellPipeline::ggsave_stream(
            "../figs/fig2B.png",
            width = 50,
            height = 20,
            units = "cm"
        )

## Fig2.C

In [ ]:
read_tsv(
    "../meta/sample_metadata.tsv",
    show_col_types = FALSE
) %>%
    distinct(sample) %>%
        pull() %>%
        as.list() %>%
        lapply(
            function(target_sample) {
                seurat <-
                    LoadH5Seurat(
                        paste0(
                            "../resources/",
                            target_sample,
                            ".h5seurat"
                        )
                    ) %>%
                    SingleCellPipeline::assign_celltype(
                        annotation = read_tsv(
                            "../meta/annotations/",
                            target_sample,
                            ".tsv"
                        ),
                        cell.meta = read_tsv(
                            "../meta/cell_metadata.tsv",
                            show_col_types = FALSE
                        ),
                        sample.meta = read_tsv(
                            "../meta/sample_metadata.tsv",
                            show_col_types = FALSE
                        )
                    )
                data <-
                    seurat@meta.data %>%
                    group_by(celltype) %>%
                    summarise(
                        n = n()
                    ) %>%
                    arrange(desc(n)) %>%
                    mutate(ypos = cumsum(n) - n / 2) %>%
                    mutate(per = n / sum(n)) %>%
                    mutate(
                        label = paste0(
                            celltype, "\n", scales::percent(per, 0.1)
                        )
                    ) %>%
                    left_join(
                        seurat@meta.data %>%
                            distinct(celltype, color.celltype),
                        by = "celltype"
                    )
                plt <-
                    data %>%
                    ggplot(
                        aes(x = 0, y = n, fill = factor(n))
                    ) +
                    geom_col() +
                    coord_polar("y") +
                    geom_text(
                        data = (. %>% filter(per >= 0.1)),
                        aes(x = 0.1, y = ypos, label = label),
                        size = 6,
                        colour = "white"
                    ) +
                    theme_void() +
                    scale_fill_manual(
                        values = data %>%
                            arrange(n) %>%
                            select(color.celltype) %>%
                            pull()
                    ) +
                    scale_color_manual(
                        values = data %>%
                            distinct(celltype) %>%
                            mutate(
                                color_text = "white"
                            ) %>%
                            select(color_text) %>%
                            pull()
                    ) +
                    ggtitle(
                        seurat@meta.data %>%
                            distinct(sample) %>%
                            pull()
                    ) +
                    theme(
                        legend.position = "none",
                        plot.title = element_text(
                            hjust = 0,
                            size = 20,
                            face = "bold"
                        ),
                        plot.background = element_rect(
                            fill = "white",
                            colour = "white"
                        )
                    )
                if (data %>%
                    filter(per < threshold) %>%
                    nrow() != 0) {
                    plt <-
                        plt +
                        geom_text_repel(
                            data = (. %>% filter(per < 0.1)),
                            aes(x = 0.4, y = ypos, label = label), nudge_x = 0.2,
                            size = 6,
                            colour = "black"
                        )
                }
                return(plt)
            }
        ) %>%
        patchwork::wrap_plots() %>%
        SingleCellPipeline::ggsave_stream(
            "../figs/fig2B.png",
            width = 50,
            height = 30,
            units = "cm"
        )
